In [1]:
import pandas as pd
import datacompy
import gc
import os
# Ouali benchmark file only includes auto aligned data
benchmark_file = 'input/rapport_20240806144726_alignement-défini_TEST ATC-_IdRef_extraction_patrinum_min2align.tsv'
abes_files = ['input/abes-alignements-surs.csv']
abes_experimental_files = ['input/abes-alignements-experimentaux-20220602.csv','input/abes-alignements-experimentaux-2efichier-methode_a-20220725.csv']
abes_noalign_file = 'input/abes-nonalign-only.csv'

set_name = 'benchmark2024-min2align'

output_folder = 'output'

# Main names for various export files
common_file = 'communs'
common_noalign_file = 'communs-nonalignements'
divergences_file = 'divergences'
div_noalign_file = 'divergences-nonalignements'
missed_file = 'manquants'

# Check that given filenames do indeed exist
all_files = []
for variable in [benchmark_file, abes_files, abes_experimental_files, abes_noalign_file]:
    if isinstance(variable, list):
        all_files.extend(variable)
    else:
        all_files.append(variable)

for file in all_files:
    if not os.path.isfile(file):
        print("ATTENTION le fichier suivant n'existe pas: ",file)

### Préparation des données Ouali

Commençons par charger les données exportées depuis Ouali en les chargeant dans des dataframes.

Pour tous les chargements de données, on s'assure que Python traite toutes les valeurs comme texte, sinon il va convertir certaines colonnes contenant des identifiants en nombres, ce qui peut poser problème par la suite si on les compare aux mêmes données chargées comme texte ou si on essaie de faire une opération join sur ces colonnes. Pour cela, on spécifie le paramètre `dtype`.

In [2]:
ouali_data = pd.read_csv(benchmark_file, sep='\t', dtype = str)
#ouali_undefined_data = pd.read_csv(benchmark_undefined_file, sep='\t', dtype = str)
# Create origin and target columns that are named the same as in the RERO files, for ease of merging
ouali_data['source'] = ouali_data['id source']
#ouali_undefined_data['source'] = ouali_undefined_data['id source']
ouali_data['cible'] = ouali_data['id cible']
# .copy() is necessary to avoid SettingWithCopyWarning when merging
ouali_align = ouali_data.query('`nombre de candidats` != "0" & `décision d\'alignement` == "auto"').copy()
ouali_no_align = ouali_data.query('`nombre de candidats` == "0" & `décision d\'alignement` == "auto"').copy()

print('Alignements Ouali chargés: ' + str(len(ouali_align)))
print('Non-alignements Ouali chargés: ' + str(len(ouali_no_align)))
#print('Alignements à arbitrer (pas de décision): ' + str(len(ouali_undefined_data)))

Alignements Ouali chargés: 293929
Non-alignements Ouali chargés: 53374


### Préparation des données ABES

Les données ABES sont chargées dans des dataframes. Pour les alignements expérimentaux, ceux-ci sont répétés lorsqu'il y a plusieurs notices bib liées. On déduplique donc ce fichier pour ne conserver qu'une ligne par alignement calculé.

In [3]:
abes_align = pd.concat((pd.read_csv(f, dtype = str) for f in abes_files))
abes_experimental_align = pd.concat((pd.read_csv(f, dtype = str) for f in abes_experimental_files ))
abes_noalign = pd.read_csv(abes_noalign_file, dtype = str)

abes_experimental_align.drop_duplicates(subset=['auth_id','IdRef_CANDIDAT'], inplace=True)
print("Nombre de concordances validées dans le fichier ABES: ",len(abes_align))
print("Nombre de concordances validées dans le fichier expérimental ABES: ",len(abes_experimental_align))
print("Nombre de non-alignements validés dans le fichier ABES: ",len(abes_noalign))

Nombre de concordances validées dans le fichier ABES:  517622
Nombre de concordances validées dans le fichier expérimental ABES:  9498
Nombre de non-alignements validés dans le fichier ABES:  380473


### Comparaison des données Ouali avec l'ABES

Dans les données ABES, l'identifiant RNV est au milieu de la chaîne de caractères présents dans la colonne `ID_EC`. Il faut donc extraire ces chiffres et les placer dans une nouvelle colonne `source` pour pouvoir les comparer à Ouali. On renomme également la colonne `IdRef` en `cible` pour faciliter la comparaison. On fait la même chose pour le fichier des non-alignements.

Les fichiers expérimentaux de l'ABES ont un autre format qu'il faut adapter. Si les fichiers expérimentaux apportent des doublons par rapport aux alignements ABES "sûrs", seuls ces derniers sont conservés.

In [4]:
abes_align['source']=abes_align['ID_EC'].str.extract(r'(\d{18})')
abes_align['cible']=abes_align['IdRef']
abes_noalign['source']=abes_noalign['ID_EC'].str.extract(r'(\d{18})')

abes_experimental_align['source']=abes_experimental_align['auth_id']
abes_experimental_align['cible']=abes_experimental_align['IdRef_CANDIDAT']
abes_experimental_align['NOM']=abes_experimental_align['auth_heading']
abes_experimental_align['NOMCANDIDAT']=abes_experimental_align['Appellation_IdRef']

all_abes = pd.concat([abes_align, abes_experimental_align])
all_abes.drop_duplicates(subset=['source'], inplace=True, keep='first')

print("Nombre de concordances ABES (valides + expérimentales): ",len(all_abes))
display(all_abes)

Nombre de concordances ABES (valides + expérimentales):  521954


,ID_EC,NB_RESU,IdRef,NOM,PRENOM,NOMCANDIDAT,PRENOMCANDIDAT,LST_HEUR,source,cible,...,Pays,Naissance,Décès,Note300,Note340,Source,Type,Type_rnv,ppn sudoc,auth_heading.1
0,renouvaud_ec_981023132827002851_bentoumi_farid,2,195225147,Bentoumi,Farid,Bentoumi,Farid,"cocontrib,titre",981023132827002851,195225147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,renouvaud_ec_981023132827302851_daniel_marie,79,134948718,Daniel,Marie,Daniel,Marie,"cocontrib,titre",981023132827302851,134948718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,renouvaud_ec_981023132828002851_boecker_virginia,1,251579700,Boecker,Virginia,Boecker,Virginia,"cocontrib,unica,titre",981023132828002851,251579700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,renouvaud_ec_981023132829502851_gonzález_ramo...,57,19625714X,González,Ramón Gilberto‏,González,R. Gilberto,"cocontrib,titre",981023132829502851,19625714X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,renouvaud_ec_981023132829802851_nadji_mehrdad,1,119903830,Nadji,Mehrdad,Nadji,Mehrdad,"cocontrib,titre",981023132829802851,119903830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44051,NaN,NaN,NaN,"Anne de Bretagne, 1477-1514, reine de France",NaN,Anne de Bretagne (1477-1514 ; reine de France),NaN,NaN,981024002093502851,027339009,...,FR,1477,1514,Duchesse de Bretagne (1488-1514) et reine de F...,Lieu de naissance Nantes. Lieu de décès Blois,Quelques lettres d'Anne de Bretagne / par M. l...,a,(RNV_A)0001629137,NaN,"Anne de Bretagne, 1477-1514, reine de France"
44052,NaN,NaN,NaN,Cani,NaN,"Cani, Bahri (journaliste)",NaN,NaN,981024002293502851,050645447,...,YU,NaN,NaN,Journaliste,NaN,"Kosmet ili Kosova / Bahri Cani, Cvijetin Miliv...",a,(RNV_A)0001629162,NaN,Cani
44072,NaN,NaN,NaN,"Zviane, 1983-",NaN,Zviane (1983-....),NaN,NaN,981024003483002851,172462177,...,CA,1983,NaN,NaN,Auteure de bandes dessinées,"L'ostie d'chat / Iris, Zviane, impr. 2012",a,(RNV_A)0001629312,NaN,"Zviane, 1983-"
44073,NaN,NaN,NaN,Leo 1944-,NaN,Léo (1944-....),NaN,NaN,981024003584702851,029470285,...,BR,19441213,NaN,NaN,Scénariste et dessinateur de bandes dessinées,"Joseph Wresinski / scénario, Dieter ; dessin, ...",a,(RNV_A)0001629337,NaN,Leo 1944-


In [5]:
common_source_abes = pd.merge(all_abes, ouali_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
common_source_noalign_abes = pd.merge(abes_noalign, ouali_no_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
print("Alignements Ouali présents dans fichier ABES: ", len(common_source_abes))

Alignements Ouali présents dans fichier ABES:  213934


In [6]:
outfile = output_folder + '/' + common_file + '-ABES-' + set_name + '.csv'
common_target_abes = pd.merge(all_abes, ouali_align, how="inner", on=['source', 'cible'], suffixes=("_abes", "_ouali"), copy=True)
common_target_abes.to_csv(outfile,columns=['source','cible','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)
print("Alignements Ouali validés par l'ABES: ", len(common_target_abes))
print("Pourcentage validé: ", round(len(common_target_abes)/len(common_source_abes)*100,1), "%")
print("Exportés dans ", outfile)

Alignements Ouali validés par l'ABES:  212146
Pourcentage validé:  99.2 %
Exportés dans  output/communs-ABES-benchmark2024-min2align.csv


In [7]:
outfile = output_folder + '/' + common_noalign_file + '-ABES-' + set_name + '.csv'
common_source_noalign_abes.to_csv(outfile,columns=['source'],encoding="UTF-8",index=False)
print("Non-alignements Ouali: ", len(ouali_no_align))
print("Non-alignements Ouali communs avec l'ABES: ", len(common_source_noalign_abes))
print("Pourcentage validé: ", round(len(common_source_noalign_abes)/len(ouali_no_align)*100,1), "%")
print("Exportés dans ", outfile)

Non-alignements Ouali:  53374
Non-alignements Ouali communs avec l'ABES:  36651
Pourcentage validé:  68.7 %
Exportés dans  output/communs-nonalignements-ABES-benchmark2024-min2align.csv


#### Différences entre Ouali et l'ABES

In [8]:
outfile = output_folder + '/' + divergences_file + '-ABES-' + set_name + '.csv'
# Alignements pour lesquels Ouali et l'ABES ont pris une décision différente (cible différente pour une même source)
divergences_abes = common_source_abes.loc[~(common_source_abes['cible_abes'] == common_source_abes['cible_ouali'])]
display(divergences_abes[['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])

# Notices qu'Ouali a alignées, mais l'ABES a pris une décision de non-alignement
divergences_NOT_abes = pd.merge(abes_noalign, ouali_align, how="inner", on=['source'], suffixes=("_abes", "_ouali"), copy=True)
divergences_NOT_abes['cible_ouali'] = divergences_NOT_abes['cible']
divergences_NOT_abes.insert(0,'cible_abes', "NaN")
display(divergences_NOT_abes[['source','cible_ouali','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])

divergences_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)
# On ajoute à ce même fichier les cas où l'ABES a pris une décision de non-alignement (mode 'a' - append)
divergences_NOT_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False, mode='a')


print("Alignements Ouali qui diffèrent dans l'ABES: ", len(divergences_abes))
print("Pourcentage avec alignement différent: ", round(len(divergences_abes)/len(common_source_abes)*100,1), "%")
print("Alignements Ouali qui sont non-alignés par l'ABES: ", len(divergences_NOT_abes))
print("Pourcentage avec décision d'alignement différente: ", round(len(divergences_NOT_abes)/len(ouali_align)*100,1), "%")
print("Exportés dans ", outfile)

,source,cible_ouali,cible_abes,forme principale cible,NOMCANDIDAT,PRENOMCANDIDAT
7,981023280468002851,030342368,090411366,"Ramadier, Jacques O. 1920-1994",Ramadier,Jacques-Alexandre
17,981023280624002851,085993859,256639787,"Rizvi, S. N. A. 19..-....",Rizvi,Athar Abbas
31,981023280966502851,128726571,092017576,"De Vries, Hendrik 1896-1989",De Vries,Hendrik
57,981023281652102851,268485011,157881865,"Michaelis, Christian Friedrich médecin 1754-1814",Michaelis,Christian Friedrich
62,981023281718302851,260913812,139698019,"Kempter, Lothar 1844-1918",Kempter,Lothar
...,...,...,...,...,...,...
213900,981023652229602851,068528787,191336408,Thomas saint,Thomas (19..-.... ; auteur pour la jeunesse),NaN
213906,981023690126802851,24821635X,082332592,Suzuka 19..-....,"Suzuka, Asaoka (19..-....)",NaN
213912,981023759225302851,257336133,032109687,Filelfo 19..-....,"Filelfo, Francesco (1398-1481)",NaN
213916,981023766605902851,251398811,075231182,"Vonaesch, Corinne 19..-....","Taddéi, Corinne",NaN


,source,cible_ouali,forme principale cible,NOMCANDIDAT,PRENOMCANDIDAT
0,981023265133302851,117619876,"Sánchez González, Juana María",NaN,NaN
1,981023280174802851,125940912,"Reymond, Henri physicien",NaN,NaN
2,981023280348402851,146466950,"Monnet, Agnieszka Soltysik 19..-....",NaN,NaN
3,981023280380802851,079311830,Jagadīśatarkālaṅkāra 16..-....,NaN,NaN
4,981023280389202851,266144292,"Trey, Daniel-Albert de 1765-1835",NaN,NaN
...,...,...,...,...,...
31165,981023878825102851,16182580X,"Keller, Peter Michael 1972-....",NaN,NaN
31166,981023879014702851,257334173,"Stumm, Anke 1987-....",NaN,NaN
31167,981023879018502851,254083005,"Joshi, Alka 19..-....",NaN,NaN
31168,981023879021402851,122844947,"Langen, Werner 1958-....",NaN,NaN


Alignements Ouali qui diffèrent dans l'ABES:  1788
Pourcentage avec alignement différent:  0.8 %
Alignements Ouali qui sont non-alignés par l'ABES:  31170
Pourcentage avec décision d'alignement différente:  10.6 %
Exportés dans  output/divergences-ABES-benchmark2024-min2align.csv


In [9]:
outfile = output_folder + '/' + div_noalign_file + '-ABES-' + set_name + '.csv'

# Non-alignements Ouali pour lesquels l'ABES a trouvé un alignement
div_nonalign_abes = pd.merge(abes_align, ouali_no_align, how="inner", on='source', suffixes=("_abes", "_ouali"), copy=True)
display(div_nonalign_abes[['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT']])

div_nonalign_abes.to_csv(outfile,columns=['source','cible_ouali','cible_abes','forme principale cible','NOMCANDIDAT', 'PRENOMCANDIDAT'],encoding="UTF-8",index=False)

print("Non-alignements Ouali erronés: ", len(div_nonalign_abes))
print("Pourcentage de divergence: ", round(len(div_nonalign_abes)/len(ouali_no_align)*100,1), "%")

print("Exportés dans ", outfile)


,source,cible_ouali,cible_abes,forme principale cible,NOMCANDIDAT,PRENOMCANDIDAT
0,981023280398402851,NaN,223370819,NaN,Albuquerque,Cristina
1,981023280800502851,NaN,076428842,NaN,Foroglou,G.
2,981023281177802851,NaN,029239664,NaN,Schilling,Diebold
3,981023281686502851,NaN,120465361,NaN,Lehmann,Heinrich
4,981023282266002851,NaN,124894259,NaN,Limberg,A A.
...,...,...,...,...,...,...
8626,981023764922702851,NaN,254832067,NaN,Künzler,Lukas
8627,981023766639502851,NaN,248191276,NaN,Flige,Irina Anatolʹevna
8628,981023769633302851,NaN,235436941,NaN,Zoboi,Ibi
8629,981023876322502851,NaN,151373094,NaN,Babic-Papadopoulos,Laurence


Non-alignements Ouali erronés:  8631
Pourcentage de divergence:  16.2 %
Exportés dans  output/divergences-nonalignements-ABES-benchmark2024-min2align.csv
